# Homework 4

In [1]:
import tensorflow as tf
import json

In [2]:
pip install horovod

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from ilsvrc_dataset import get_datasets

--------------------------------------------------------------------------

  Local host:   thetagpu06
  Local device: mlx5_0
--------------------------------------------------------------------------


In [4]:
class FakeHvd:
    
    def size(self): return 1
    
    def rank(self): return 0


with open("ilsvrc.json", 'r') as f: 
    config = json.load(f)

print(json.dumps(config, indent=4))
    
    
config['hvd'] = FakeHvd()

{
    "data": {
        "handler": "ilsvrc_dataset",
        "batch_size": 128,
        "train_filelist": "/lus/grand/projects/ALCFAITP/ilsvrc_train_filelist.txt",
        "test_filelist": "/lus/grand/projects/ALCFAITP/ilsvrc_val_filelist.txt",
        "shuffle_buffer": 200000,
        "reshuffle_each_iteration": true,
        "num_parallel_readers": 32,
        "prefetch_buffer_size": 10,
        "crop_image_size": [
            256,
            256
        ],
        "num_classes": 1000,
        "num_channels": 3
    }
}


In [5]:
train_ds, val_ds = get_datasets(config)

num labels: 1000
build dataset /lus/grand/projects/ALCFAITP/ilsvrc_train_filelist.txt
input filelist contains 1281167 files, estimated batches per rank 10009
build dataset /lus/grand/projects/ALCFAITP/ilsvrc_val_filelist.txt
input filelist contains 50000 files, estimated batches per rank 390


In [6]:
images, labels = next(iter(train_ds.take(1)))

In [7]:
print(images.shape)
print(labels.shape)

(128, 256, 256, 3)
(128,)


In [8]:
class DenseLayer(tf.keras.Model):
    
    def __init__(self, n_filters):
        tf.keras.Model.__init__(self)
        
        self.conv1 = tf.keras.layers.Conv2D(
            filters     = n_filters,
            kernel_size = (3,3),
            padding     = "same"
        )
        
    def __call__(self, inputs):
        
        x = inputs
        
        output1 = self.conv1(inputs)
        
        output1 = tf.keras.activations.sigmoid(output1)
        
        return output1
    
class ResidualLayer(tf.keras.Model):
    
    def __init__(self, n_filters):
        tf.keras.Model.__init__(self)
        
        self.conv1 = tf.keras.layers.Conv2D(
            filters     = n_filters,
            kernel_size = (3,3),
            padding     = "same"
        )
        
        self.conv2 = tf.keras.layers.Conv2D(
            filters     = n_filters,
            kernel_size = (3,3),
            padding     = "same"
        )
    
    def __call__(self, inputs):
        
        x = inputs
        
        output1 = self.conv1(inputs)
        
        output1 = tf.keras.activations.sigmoid(output1)
        
        output2 = self.conv2(output1)

        return tf.keras.activations.sigmoid(output2 + x)
        